In [162]:
import json
import pandas as pd
import re
import os

In [163]:
with open('dataset.json','rb') as json_file:
    dataset = json.load(json_file)
codeforces_dataset = list()
for problem in dataset:
    if ('codeforces' in problem['source']):
        for solution in problem['solutions']:
            if (solution['code'] != None and problem['index']):
                codeforces_dataset.append({'problem':problem['index'],'code_name' : problem['index'], 'code_content': solution['code'],'tags':problem['tags']})

In [164]:
class Code_Preprocessing():
    def __init__(self,text):
        self.text = text
    def lower_text(self):
        return self.text.lower()
    def comment_remover(self):
        def replacer(match):
            s = match.group(0)
            if s.startswith('/'):
                return " " # note: a space and not an empty string
            else:
                return s
        pattern = re.compile(
            r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
            re.DOTALL | re.MULTILINE)
        return re.sub(pattern, replacer, self.text)
    def clean_space_line(self):
        self.text = self.text.replace('\n',' ')
        self.text = ' '.join(self.text.split())
        return self.text
    def clean(self):
        self.text = self.lower_text()
        self.text = self.comment_remover()
        self.text = self.clean_space_line()
        return self.text

In [165]:
codeforces_dataset = pd.DataFrame(data = codeforces_dataset)
for index in range(len(codeforces_dataset['code_content'])):
    reg = re.match('(\d+)(\w+)',codeforces_dataset['problem'][index])
    codeforces_dataset['problem'][index] = reg.groups()[0] + '_' + reg.groups()[1]
    codeforces_dataset['code_content'][index] = Code_Preprocessing(codeforces_dataset['code_content'][index]).clean()
codeforces_dataset.head()

,problem,code_name,code_content,tags
0,1366_G,1366G,#include<cstdio> #include<cstdlib> #include<cs...,"[dp & greedy, dp, data structures, strings]"
1,1366_G,1366G,#include<bits/stdc++.h> #define f first #defin...,"[dp & greedy, dp, data structures, strings]"
2,1366_G,1366G,#include <bits/stdc++.h> using namespace std; ...,"[dp & greedy, dp, data structures, strings]"
3,1366_F,1366F,#include <bits/stdc++.h> using namespace std; ...,"[geometry, binary search, dp, graphs, dp & gre..."
4,1366_F,1366F,#include <bits/stdc++.h> #define f first #defi...,"[geometry, binary search, dp, graphs, dp & gre..."


In [166]:
remove_list = list()
for index in range(len(codeforces_dataset['problem'])):
    if (codeforces_dataset['problem'][index] not in os.listdir('code_forces_6_3')):
        remove_list.append(codeforces_dataset['problem'][index])

In [167]:
dataset = codeforces_dataset[codeforces_dataset['problem'].isin(remove_list) == True]

In [168]:
path = 'add_dataset/'
num = 0
for index in range(len(dataset)):
    if os.path.exists(path + dataset['problem'].iloc[index]) == False:
            os.mkdir(path + dataset['problem'].iloc[index])
    with open(path + dataset['problem'].iloc[index] + '/' + str(num) + '.cpp', 'w') as cpp_file:
        cpp_file.write(dataset['code_content'].iloc[index])
    num+=1

In [169]:
print(type(dataset['problem'].iloc[0]))

<class 'str'>
